In [1]:
# baseline cnn model for mnist
from numpy import mean
from numpy import std
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
import tensorflow as tf

In [2]:
from keras.preprocessing.image import ImageDataGenerator

In [3]:

folder_test = '../archive/test/'
dataset_test = tf.keras.utils.image_dataset_from_directory(folder_test, image_size=(224, 224), batch_size=1)

folder_train = '../archive/train/'
dataset_train = tf.keras.utils.image_dataset_from_directory(folder_train, image_size=(224, 224), batch_size=2)

folder_valid = '../archive/valid/'
dataset_valid = tf.keras.utils.image_dataset_from_directory(folder_valid, image_size=(224, 224), batch_size=1)

Found 2625 files belonging to 525 classes.
Found 84635 files belonging to 525 classes.
Found 2625 files belonging to 525 classes.


In [4]:
dict_class_labels = {j:i for i,j in enumerate(dataset_train.class_names)}
len(dict_class_labels)

525

In [5]:
print(dict_class_labels)

{'ABBOTTS BABBLER': 0, 'ABBOTTS BOOBY': 1, 'ABYSSINIAN GROUND HORNBILL': 2, 'AFRICAN CROWNED CRANE': 3, 'AFRICAN EMERALD CUCKOO': 4, 'AFRICAN FIREFINCH': 5, 'AFRICAN OYSTER CATCHER': 6, 'AFRICAN PIED HORNBILL': 7, 'AFRICAN PYGMY GOOSE': 8, 'ALBATROSS': 9, 'ALBERTS TOWHEE': 10, 'ALEXANDRINE PARAKEET': 11, 'ALPINE CHOUGH': 12, 'ALTAMIRA YELLOWTHROAT': 13, 'AMERICAN AVOCET': 14, 'AMERICAN BITTERN': 15, 'AMERICAN COOT': 16, 'AMERICAN DIPPER': 17, 'AMERICAN FLAMINGO': 18, 'AMERICAN GOLDFINCH': 19, 'AMERICAN KESTREL': 20, 'AMERICAN PIPIT': 21, 'AMERICAN REDSTART': 22, 'AMERICAN ROBIN': 23, 'AMERICAN WIGEON': 24, 'AMETHYST WOODSTAR': 25, 'ANDEAN GOOSE': 26, 'ANDEAN LAPWING': 27, 'ANDEAN SISKIN': 28, 'ANHINGA': 29, 'ANIANIAU': 30, 'ANNAS HUMMINGBIRD': 31, 'ANTBIRD': 32, 'ANTILLEAN EUPHONIA': 33, 'APAPANE': 34, 'APOSTLEBIRD': 35, 'ARARIPE MANAKIN': 36, 'ASHY STORM PETREL': 37, 'ASHY THRUSHBIRD': 38, 'ASIAN CRESTED IBIS': 39, 'ASIAN DOLLARD BIRD': 40, 'ASIAN GREEN BEE EATER': 41, 'ASIAN OPENBILL

In [7]:
data_imagen = []
label_imagen = []
var = 0
for images, labels  in dataset_train:
    data_imagen.append(images.numpy())
    label_imagen.append(labels[0].numpy())

: 

Preper data for model

In [7]:
def define_model():
    """Convolutional NN model creation
    """
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
	
    # compile model
    opt = SGD(lr=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

def evaluate_model(dataX, dataY, n_folds=5):
    """Evaluate a model using k-fold cross-validation
    """
    scores, histories = list(), list()
    # prepare cross validation
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    
    # enumerate splits
    for train_ix, test_ix in kfold.split(dataX):
        # define model
        model = define_model()
        # select rows for train and test
        trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
        # fit model
        history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
        # evaluate model
        _, acc = model.evaluate(testX, testY, verbose=0)
        print('> %.3f' % (acc * 100.0))

        # stores scores
        scores.append(acc)
        histories.append(history)
        
    return scores, histories

In [31]:
def run_test_harness():
    # evaluate model
    scores, histories = evaluate_model(data_imagen, label_imagen)
    
    return scores, histories 
# entry point, run the test harness
scores, histories = run_test_harness()

2023-11-08 15:25:19.647782: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


ValueError: in user code:

    File "/usr/local/Caskroom/miniforge/base/envs/compu-vision/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/Caskroom/miniforge/base/envs/compu-vision/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/Caskroom/miniforge/base/envs/compu-vision/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/Caskroom/miniforge/base/envs/compu-vision/lib/python3.9/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/Caskroom/miniforge/base/envs/compu-vision/lib/python3.9/site-packages/keras/engine/training.py", line 1082, in compute_loss
        return self.